# 8. [**Calculer avec scikit-learn**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/8_computing.ipynb)</br>([*Computing with scikit-learn*](https://scikit-learn.org/stable/computing.html))

✔ 8.1. [**Stratégies de mise à l'échelle informatique : données plus volumineuses**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/8_computing.ipynb#strategies-to-scale-computationally-bigger-data)
([*Strategies to scale computationally: bigger data*](https://scikit-learn.org/stable/computing.html#strategies-to-scale-computationally-bigger-data))

* ✔ 8.1.1. [**Mise à l'échelle avec des instances utilisant l'apprentissage hors cœur**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/8_computing.ipynb#scaling-with-instances-using-out-of-core-learning)
([*Scaling with instances using out-of-core learning*](https://scikit-learn.org/stable/computing.html#scaling-with-instances-using-out-of-core-learning))

8.2. [**Performances informatiques**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/8_computing.ipynb#)
([*Computational Performance*](https://scikit-learn.org/stable/computing.html#))

* 8.2.1. [**Latence de prédiction**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/8_computing.ipynb#)
([*Prediction Latency*](https://scikit-learn.org/stable/computing.html#))
* 8.2.2. [**Débit de prédiction**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/8_computing.ipynb#)
([*Prediction Throughput*](https://scikit-learn.org/stable/computing.html#))
* 8.2.3. [**Trucs et astuces**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/8_computing.ipynb#)
([*Tips and Tricks*](https://scikit-learn.org/stable/computing.html#))

8.3. [**Parallélisme, gestion des ressources et configuration**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/8_computing.ipynb#)
([*Parallelism, resource management, and configuration*]())

* 8.3.1. [**Parallélisme**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/8_computing.ipynb#)
([*Parallelism*]())
* 8.3.2. [**Commutateurs de configuration**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/8_computing.ipynb#)
([*Configuration switches*]())

<a id='strategies-to-scale-computationally-bigger-data'></a>

# 8.1. Stratégies de mise à l'échelle informatique : données plus volumineuses

Pour certaines applications, la quantité d'exemples, de caractéristiques (ou les deux) et/ou la vitesse à laquelle ils doivent être traités sont difficiles pour les approches traditionnelles. Dans ces cas, scikit-learn propose un certain nombre d'options que vous pouvez envisager pour faire évoluer votre système.

<a id='scaling-with-instances-using-out-of-core-learning'></a>

## 8.1.1. Mise à l'échelle avec des instances utilisant l'apprentissage hors cœur

L'apprentissage hors cœur (ou "mémoire externe") est une technique utilisée pour apprendre à partir de données qui ne peuvent pas tenir dans la mémoire principale (RAM) d'un ordinateur.

Voici une esquisse d'un système conçu pour atteindre cet objectif :
1. un moyen de diffuser des instances
2. un moyen d'extraire des caractéristiques à partir d'instances
3. un algorithme incrémental

<a id='streaming-instances'></a>

### 8.1.1.1. Instances de diffusion en continu

Fondamentalement, 1. peut être un lecteur qui produit des instances à partir de fichiers sur un disque dur, une base de données, à partir d'un flux réseau, etc. Cependant, les détails sur la façon d'y parvenir dépassent le cadre de cette documentation.

<a id='extracting-features'></a>

### 8.1.1.2. Extraction de caractéristiques

Le 2. pourrait être n'importe quel moyen pertinent d'extraire des caractéristiques parmi les différentes méthodes d'[extraction de caractéristiques (6.2.)](https://scikit-learn.org/stable/modules/feature_extraction.html) prises en charge par scikit-learn. Cependant, lorsque vous travaillez avec des données nécessitant une vectorisation et lorsque l'ensemble de caractéristiques ou de valeurs n'est pas connu à l'avance, vous devez faire particulièrement attention. Un bon exemple est la classification de texte où des termes inconnus sont susceptibles d'être trouvés pendant l'entraînement. Il est possible d'utiliser un vectoriseur avec état si effectuer plusieurs passages sur les données est raisonnable d'un point de vue applicatif. Sinon, on peut augmenter la difficulté en utilisant un extracteur de caractéristiques sans état. Actuellement, la méthode préférée consiste à utiliser la prétendue [astuce de hachage (6.2.2.)](https://scikit-learn.org/stable/modules/feature_extraction.html#feature-hashing) telle qu'implémentée par [`sklearn.feature_extraction.FeatureHasher`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.FeatureHasher.html#sklearn.feature_extraction.FeatureHasher) pour les ensembles de données avec des variables catégorielles représentées sous forme de liste de dicts Python ou [`sklearn.feature_extraction.text.HashingVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer) pour les documents texte.

<a id='incremental-learning'></a>

### 8.1.1.3. Apprentissage incrémental

Enfin, pour 3. nous avons un certain nombre d'options dans scikit-learn. Bien que tous les algorithmes ne puissent pas apprendre de manière incrémentielle (c'est-à-dire sans voir toutes les instances à la fois), tous les estimateurs implémentant l'API `partial_fit` sont candidats. En fait, la capacité d'apprendre progressivement à partir d'un mini-lot d'instances (parfois appelé "apprentissage en ligne") est la clé de l'apprentissage hors cœur car il garantit qu'à un moment donné, il n'y aura qu'un petit nombre d'instances dans le mémoire principale. Le choix d'une bonne taille pour le mini-lot qui équilibre la pertinence et l'empreinte mémoire peut impliquer quelques ajustements [1].

Voici une liste d'estimateurs incrémentaux pour différentes tâches :

#### Classification
* [`sklearn.naive_bayes.MultinomialNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB)
* [`sklearn.naive_bayes.BernoulliNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB)
* [`sklearn.linear_model.Perceptron`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html#sklearn.linear_model.Perceptron)
* [`sklearn.linear_model.SGDClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier)
* [`sklearn.linear_model.PassiveAggressiveClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.PassiveAggressiveClassifier.html#sklearn.linear_model.PassiveAggressiveClassifier)
* [`sklearn.neural_network.MLPClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)

#### Régression
* [`sklearn.linear_model.SGDRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor)
* [`sklearn.linear_model.PassiveAggressiveRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.PassiveAggressiveRegressor.html#sklearn.linear_model.PassiveAggressiveRegressor)
* [`sklearn.neural_network.MLPRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor)

#### Regroupement
* [`sklearn.cluster.MiniBatchKMeans`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html#sklearn.cluster.MiniBatchKMeans)
* [`sklearn.cluster.Birch`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html#sklearn.cluster.Birch)

#### Décomposition / Extraction de caractéristiques
* [`sklearn.decomposition.MiniBatchDictionaryLearning`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.MiniBatchDictionaryLearning.html#sklearn.decomposition.MiniBatchDictionaryLearning)
* [`sklearn.decomposition.IncrementalPCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.IncrementalPCA.html#sklearn.decomposition.IncrementalPCA)
* [`sklearn.decomposition.LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#sklearn.decomposition.LatentDirichletAllocation)
* [`sklearn.decomposition.MiniBatchNMF`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.MiniBatchNMF.html#sklearn.decomposition.MiniBatchNMF)

#### Prétraitement
* [`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler)
* [`sklearn.preprocessing.MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler)
* [`sklearn.preprocessing.MaxAbsScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html#sklearn.preprocessing.MaxAbsScaler)

Pour la classification, une chose quelque peu importante à noter est que bien qu'une routine d'extraction de caractéristiques sans état puisse être capable de faire face à des attributs nouveaux/indédits, l'apprenant incrémental lui-même peut être incapable de faire face à des classes cibles nouvelles/indédites. Dans ce cas, vous devez passer toutes les classes possibles au premier appel de `partial_fit` en utilisant le paramètre `classes=`.

Un autre aspect à considérer lors du choix d'un algorithme approprié est que tous n'accordent pas la même importance à chaque exemple au fil du temps. A savoir, le `Perceptron` est toujours sensible aux exemples mal étiquetés même après de nombreux exemples alors que les familles `SGD*` et `PassiveAggressive*` sont plus robustes à ce genre d'artefacts. À l'inverse, ces derniers ont également tendance à accorder moins d'importance à des exemples remarquablement différents, mais correctement étiquetés, lorsqu'ils arrivent tard dans le flux, car leur taux d'apprentissage diminue avec le temps.

### 8.1.1.4. Exemples

Enfin, nous avons un exemple complet de [classification hors cœur de documents texte](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/plot_out_of_core_classification.ipynb). Il vise à fournir un point de départ aux personnes souhaitant créer des systèmes d'apprentissage hors cœur et illustre la plupart des notions abordées ci-dessus.

De plus, il montre également l'évolution des performances des différents algorithmes avec le nombre d'exemples traités.

<img alt="accuracy_over_time" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_out_of_core_classification_001.png" style="width: 512.0px; height: 384.0px;" />

En regardant maintenant le temps de calcul des différentes parties, on s'aperçoit que la vectorisation est bien plus coûteuse que l'apprentissage lui-même. Parmi les différents algorithmes, `MultinomialNB` est le plus cher, mais sa surcharge peut être atténuée en augmentant la taille des mini-lots (exercice : changez `minibatch_size` en 100 et 10000 dans le programme et comparez).

<img alt="computation_time" src="https://scikit-learn.org/stable/_images/sphx_glr_plot_out_of_core_classification_003.png" style="width: 512.0px; height: 384.0px;" />

### 8.1.1.5. Remarques

[1] Selon l'algorithme, la taille du mini-lot peut influencer ou non les résultats. `SGD*`, `PassiveAggressive*` et les NaiveBayes discrets sont véritablement en ligne et ne sont pas affectés par la taille du lot. Inversement, le taux de convergence de MiniBatchKMeans est affecté par la taille du lot. En outre, son empreinte mémoire peut varier considérablement en fonction de la taille du lot.